In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

# Load Yelp dataset
df = pd.read_csv('./Yelp_Reviews.csv', usecols=[3])
data = df.values.tolist()

# Cluster the encoded data using KMeans or other clustering algorithm

# Use the clusters as input to a supervised learning model for sentiment classification


In [2]:
# Preprocess data
max_len = 200
num_samples = len(data)

input_texts = []
target_texts = []
input_chars = set()

for i in range(num_samples):
    input_text = data[i][0][:max_len]
    target_text = data[i][0][:max_len]
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)

input_chars = sorted(list(input_chars))
num_encoder_tokens = len(input_chars)
max_encoder_seq_length = max_len

input_token_index = dict((c, i) for i, c in enumerate(input_chars))
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(input_texts):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

In [3]:
# Define the autoencoder LSTM architecture
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim)(encoder_inputs)

decoder_inputs = RepeatVector(max_encoder_seq_length)(encoder)
decoder = LSTM(num_encoder_tokens, return_sequences=True)(decoder_inputs)

autoencoder = Model(encoder_inputs, decoder)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [4]:
# Train the autoencoder on Yelp dataset
epochs = 50
batch_size = 64

autoencoder.fit(encoder_input_data, encoder_input_data, batch_size=batch_size, epochs=epochs)

Epoch 1/50
41/41 [==============================] - 20s 429ms/step - loss: 0.0701
Epoch 2/50
41/41 [==============================] - 18s 438ms/step - loss: 0.0701
Epoch 3/50
41/41 [==============================] - 18s 449ms/step - loss: 0.0701
Epoch 4/50
41/41 [==============================] - 18s 442ms/step - loss: 0.0701
Epoch 5/50
41/41 [==============================] - 18s 442ms/step - loss: 0.0701
Epoch 6/50
41/41 [==============================] - 18s 447ms/step - loss: 0.0701
Epoch 7/50
41/41 [==============================] - 18s 448ms/step - loss: 0.0701
Epoch 8/50
41/41 [==============================] - 18s 450ms/step - loss: 0.0701
Epoch 9/50
41/41 [==============================] - 19s 452ms/step - loss: 0.0701
Epoch 10/50
41/41 [==============================] - 19s 470ms/step - loss: 0.0701
Epoch 11/50
41/41 [==============================] - 19s 460ms/step - loss: 0.0701
Epoch 12/50
41/41 [==============================] - 19s 453ms/step - loss: 0.0701
Epoch 13/50
4

In [5]:
# Use the trained autoencoder for sentiment analysis
encoder_model = Model(encoder_inputs, encoder)
encoder_model.compile(optimizer='adam', loss='binary_crossentropy')

In [6]:
# Encode Yelp reviews
encoded_data = encoder_model.predict(encoder_input_data)

82/82 [==============================] - 9s 101ms/step
